In [186]:
import pandas as pd
import math

import os
from fnmatch import fnmatch


col = ['Tariff name', 'LLFCs', 'Red/black unit charge\np/kWh',
       'Amber/yellow unit charge\np/kWh', 'Green unit charge\np/kWh',
       'Fixed charge p/MPAN/day', 'Capacity charge p/kVA/day', 'Distributor Short Code (GSP)', 'Market Participant ID']
joined_df = pd.DataFrame(columns = col)

In [187]:
root = r'C:\Users\ThomasChow\URBANCHAIN LTD\Shared Materials - Materials\Operations\001 Pricing\DUoS\Viktor DUOS charging statements\2024-2025 UPDATED'

mapping = pd.read_csv(root + '/Market_Participant_role_353.csv')

# mapping
mapping = mapping[mapping['Distributor Short Code'].isna() == False]
mapping = mapping.astype({'Distributor Short Code' : 'int'})

map_dict = pd.Series(mapping['Market Participant ID'].values,index=mapping['Distributor Short Code']).to_dict()
map_dict

{42: 'AENL',
 46: 'AIDI',
 77: 'AMSN',
 45: 'AURO',
 97: 'CIDA',
 98: 'CIDB',
 99: 'CIDC',
 28: 'EDFI',
 10: 'EELC',
 11: 'EMEB',
 27: 'ETCL',
 35: 'FEAL',
 37: 'FORB',
 33: 'GGEN',
 26: 'GUCL',
 29: 'HARL',
 17: 'HYDE',
 43: 'IDCS',
 38: 'INDI',
 24: 'IPNL',
 25: 'LENG',
 12: 'LOND',
 13: 'MANW',
 14: 'MIDE',
 34: 'MPDL',
 15: 'NEEB',
 16: 'NORW',
 30: 'PENL',
 19: 'SEEB',
 20: 'SOUT',
 18: 'SPOW',
 39: 'STRK',
 21: 'SWAE',
 22: 'SWEB',
 32: 'UDNL',
 31: 'UKPD',
 40: 'UTAL',
 36: 'VATT',
 23: 'YELG'}

In [188]:
import re

def match_text(text):

    matches = re.findall(r'GSP[_ ]?\d+', text)

# Output the result
    return int(matches[0].lstrip('GSP_'))


In [189]:
root = r'C:\Users\ThomasChow\URBANCHAIN LTD\Shared Materials - Materials\Operations\001 Pricing\DUoS\Viktor DUOS charging statements\2024-2025 UPDATED'
pattern = "*.xlsx"



for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):

            f = os.path.join(path, name)
            
            xls = pd.ExcelFile(f)
            names = [n for n in xls.sheet_names if 'Annex 1' in n]

            for n in names:
                source = pd.read_excel(f, sheet_name = n)

                # Find where the record starts
                skip = source[source.isin(['Tariff name'])].first_valid_index() + 1
                df = source.copy()

                # Drop the redundant rows
                df = df.iloc[skip-1:].reset_index(drop = True).T.set_index(0).T
                df.head()

                df['Distributor Short Code (GSP)'] = match_text(f.lstrip(root))
                df['Market Participant ID'] = df['Distributor Short Code (GSP)'].apply(lambda x: map_dict[x]) 

                # Remove NA values
                df = df.fillna('')
                open = df.columns[1]
                


                df = df.astype({open: 'string', 'Closed LLFCs': 'string'})

                df['LLFCs'] = df[open] + ', ' + df['Closed LLFCs']
                df['LLFCs'] = df['LLFCs'].apply(lambda x: x.rstrip(', '))

                for p in [', ', ',', '-', '.', ' ']:  
                    df['LLFCs'] = df['LLFCs'].str.split(p)
                    df = df.explode('LLFCs')[col].reset_index(drop = True)

                df = df.drop(df[df['Red/black unit charge\np/kWh'] == ''].index)
                joined_df = pd.concat([joined_df, df])



In [190]:
joined_df

,Tariff name,LLFCs,Red/black unit charge\np/kWh,Amber/yellow unit charge\np/kWh,Green unit charge\np/kWh,Fixed charge p/MPAN/day,Capacity charge p/kVA/day,Distributor Short Code (GSP),Market Participant ID
0,Domestic Aggregated or CT with Residual,1,15.296,1.203,0.246,13.5,,10,EELC
1,Domestic Aggregated or CT with Residual,3,15.296,1.203,0.246,13.5,,10,EELC
2,Domestic Aggregated or CT with Residual,7,15.296,1.203,0.246,13.5,,10,EELC
3,Domestic Aggregated (Related MPAN),2,15.296,1.203,0.246,,,10,EELC
4,Domestic Aggregated (Related MPAN),22,15.296,1.203,0.246,,,10,EELC
...,...,...,...,...,...,...,...,...,...
95,LV Sub Generation Site Specific,,-6.282,-2.08,-0.507,0,,38,INDI
96,LV Sub Generation Site Specific no RP charge,,-6.282,-2.08,-0.507,0,,38,INDI
97,HV Generation Site Specific,PR4,-3.437,-0.847,-0.311,767.21,,38,INDI
98,HV Generation Site Specific,PV4,-3.437,-0.847,-0.311,767.21,,38,INDI


In [191]:
joined_df.to_csv(r'C:\Users\ThomasChow\OneDrive - URBANCHAIN LTD\Documents\Data Cleaning\duos.csv', index = False)

In [192]:
# df = pd.read_csv(r"C:\Users\ThomasChow\OneDrive - URBANCHAIN LTD\Documents\Data Cleaning\DUOS.csv")

# def convert_range(x):
#     if isinstance(x, str):
#         if "-" in x:
#             first = ''
#             if x[0].isalpha():
#                 first = x[0]
#                 x = x[1:3]
#             start = int(x.split('-')[0])
#             end = int(x.split('-')[0]) + 1
#             r = range(start, end)
#             print(list(r))
#             x = [f'{first}{x}' for x in list(r)]
#     return x
        
# df.LLFCs = df.LLFCs.apply(lambda x: convert_range(x))
# df = df.explode('LLFCs')[col].reset_index(drop = True)
# df.to_csv(r"C:\Users\ThomasChow\OneDrive - URBANCHAIN LTD\Documents\Data Cleaning\DUOS.csv", index = False)

In [3]:
import pandas as pd
df = pd.read_csv(r"C:\Users\ThomasChow\OneDrive - URBANCHAIN LTD\Documents\Data Cleaning\duos.csv")
df[df.LLFCs == "1E1"]

,Tariff name,LLFCs,Red/black unit charge\np/kWh,Amber/yellow unit charge\np/kWh,Green unit charge\np/kWh,Fixed charge p/MPAN/day,Capacity charge p/kVA/day,Distributor Short Code (GSP),Market Participant ID
21405,Non-Domestic Aggregated or CT Band 1,1E1,9.704,1.598,0.237,18.04,0.0,35,FEAL


In [9]:
df.to_csv(r"C:\Users\ThomasChow\OneDrive - URBANCHAIN LTD\Documents\Data Cleaning\duos.csv", index = False)